In [1]:
from scipy.interpolate import lagrange
x_values = [1, 2, 3, 4]
y_values = [4, 8, 2, 1]

print(lagrange(x_values, y_values))
#      3      2
# 2.5 x - 20 x + 46.5 x - 25

     3      2
2.5 x - 20 x + 46.5 x - 25


## The above in finite field

Let’s use the same polynomial as before, but this time we’ll use a finite field $\mathbb{F}_{17}$ instead of floating point numbers.

In [6]:
import galois
import numpy as np
GF17 = galois.GF(17)

xs = GF17(np.array([1,2,3,4]))
ys = GF17(np.array([4,8,2,1]))

p = galois.lagrange_poly(xs, ys)

print(p)

print(p(1), p(2), p(3), p(4))

assert p(1) == GF17(4)
assert p(2) == GF17(8)
assert p(3) == GF17(2)
assert p(4) == GF17(1)

11x^3 + 14x^2 + 4x + 9
4 8 2 1


**For a set of $n$ points, there is a unique lowest-degree polynomial of at most degree $n - 1$ that interpolates them.**  
The conosequence of this is that  
**If we use the points $(1,2,…,n)$ as the $x$ values to convert a length $n$ vector to a polynomial via Lagrange interpolation, then the resulting polynomial is unique.**

# The Schwartz-Zippel Lemma and its application to Zero Knowledge Proofs

Nearly all ZK-Proof algorithms rely on the Schwartz-Zippel Lemma to achieve succintness. 

The Schwartz-Zippel Lemma states that if we are given two polynomials $p(x)$ and $q(x)$ with degrees $d_p$ and $d_q$ respectively, and if $p(x) \neq q(x)$, then the number of points where $p(x)$ and $q(x)$ intersect is less than or equal to $max(d_p, d_q)$.

## Polynomials in finite fields and the Schwartz-Zippel Lemma

### Polynomial equality testing

We can test that two polynomials are equal by checking if all their coefficients are equal, but this takes $\mathcal{O}(d)$ time, where $d$ is the degree of the polynomial. If instead we can evaluate the polynomials at a random point $u$ and compare the evaluations $\mathcal{O}(1)$ in time.  
So, in finite field $\mathbb{F}_p$, $u \in [0,p)$ and we check for $f(x) \stackrel{?}{=} g(x)$. For large enough $p$, like $p \approx 2^{254}$, this equality check tell if both ply are equal or not with maximum certinity (and ignoring that the point chosen is just an intersecting point).  

### Using the Schwartz-Zippel Lemma to test if two vectors are equal

We can combine Lagrange interpolation with the Schwartz-Zippel Lemma to test if two vectors are equal. We use common values of $x$ (say $[1,2,3, ..., n]) to interpolate the vectors (the y values are the coeffecients or "columns" of the vectors). So we do:
1. Interpolate a polynomial for each vector $f(x)$ and $g(x)$
2. Pick a random $u$
3. Evaluate the polynomial as $u$
4. Check if $f(x) = g(x)$

Below is an example in python.

In [1]:
import galois
import numpy as np

p = 103
GF = galois.GF(p)

# We take 3 values as we have 3 rows in the vector
xs = GF(np.array([1,2,3]))

# arbitrary vectors
v1 =  GF(np.array([4,8,19]))
v2 =  GF(np.array([4,8,19]))


def L(v):
    return galois.lagrange_poly(xs, v)

p1 = L(v1)
p2 = L(v2)

import random
u = random.randint(0, p)

lhs = p1(u)
rhs = p2(u)

# only one check required
assert lhs == rhs